In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect,Integer

# Create an engine that can talk to the database
engine = create_engine("sqlite:///./Resources/sqlite/Formula1_4tables.sqlite")

# Explore Database
inspector = inspect(engine)
print("Working with the following tables:  ")
print(inspector.get_table_names())
print()

# Create a function that will help with creating data frame for each table
def create_df(data, table_name):
    columns = inspector.get_columns(table_name)
    # Get the column names and types
    # List to hold the names for columns
    col_names = []
    for c in columns:
        # print(c["name"],c["type"])
        col_names.append(c["name"])
    df = pd.DataFrame(data, columns=col_names)
    return df
    

# Query the info for circuits, drivers, races and results
circuit_data = engine.execute("SELECT * FROM circuits")
drivers_data = engine.execute("SELECT * FROM drivers")
races_data = engine.execute("SELECT * FROM races")
results_data = engine.execute("SELECT * FROM results")

# Query All Records in the the Database and make a DF out of the table
circuit_df = create_df(circuit_data, "circuits")
drivers_df = create_df(drivers_data, "drivers")
races_df = create_df(races_data, "races")
results_df = create_df(results_data, "results")
results_df.head()

Working with the following tables:  
['circuits', 'drivers', 'races', 'results']



,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,34:50.6,5690616,39,2,01:27.5,218.3,1
1,2,18,2,2,3,5,2,2,2,8.0,58,5.478,5696094,41,3,01:27.7,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,8.163,5698779,41,5,01:28.1,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,17.181,5707797,58,7,01:28.6,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,18.014,5708630,43,1,01:27.4,218.385,1


In [2]:
# The goal of the project is to look at the top 3 best drivers for each year starting from 2014 to 2017
# we need to clean the data based on this goal
# clean circuits data
new_circ_df = circuit_df[["circuitId","name","country"]]

#clean drivers data
new_drivers_df = drivers_df[["driverId","forename","surname","nationality"]]

#clean race data
new_race_df = races_df[["raceId","year","circuitId","name"]]

#clean results data
new_results_df = results_df[["raceId","driverId","constructorId","points","positionOrder"]]


In [3]:
# Merge the results and driver data
driver_result_df = pd.merge(new_drivers_df, new_results_df, on="driverId")
driver_result_df
# Merge the circuits and races data
circ_race_df = pd.merge(new_circ_df, new_race_df,  on="circuitId")


In [4]:
circ_race_df.head()

,circuitId,name_x,country,raceId,year,name_y
0,1,Albert Park Grand Prix Circuit,Australia,1,2009,Australian Grand Prix
1,1,Albert Park Grand Prix Circuit,Australia,18,2008,Australian Grand Prix
2,1,Albert Park Grand Prix Circuit,Australia,36,2007,Australian Grand Prix
3,1,Albert Park Grand Prix Circuit,Australia,55,2006,Australian Grand Prix
4,1,Albert Park Grand Prix Circuit,Australia,71,2005,Australian Grand Prix


In [5]:
# Droping one column that repeats
# the columns are the same, but one column has more detailed name, when merging it creates two in stead of one of this
circ_race_df = circ_race_df.drop(columns=["name_y"])
circ_race_df = circ_race_df.rename(columns={"name_x": "name"})
circ_race_df

,circuitId,name,country,raceId,year
0,1,Albert Park Grand Prix Circuit,Australia,1,2009
1,1,Albert Park Grand Prix Circuit,Australia,18,2008
2,1,Albert Park Grand Prix Circuit,Australia,36,2007
3,1,Albert Park Grand Prix Circuit,Australia,55,2006
4,1,Albert Park Grand Prix Circuit,Australia,71,2005
...,...,...,...,...,...
992,71,Sochi Autodrom,Russia,972,2017
993,71,Sochi Autodrom,Russia,1004,2018
994,73,Baku City Circuit,Azerbaijan,955,2016
995,73,Baku City Circuit,Azerbaijan,976,2017


In [6]:
# Merge all data together
formula1_df = pd.merge(driver_result_df, circ_race_df, on="raceId")

# Create an empty data frame
f1_df = pd.DataFrame()

# We are only intersted to find the data since 1950 to 2017
for year in range(1950, 2018):
    formula1_year = formula1_df[formula1_df["year"]==year]
    
    # Group by year, driver ID, and take the the surname, forname and nationality
    # take the sum of drivers points of that year
    # and then after sorting them, take the first 3 places
    points_place_year = formula1_year.groupby(["year","driverId","constructorId","forename","surname","nationality"]).sum()["points"]
    points_place_year  = points_place_year.sort_values(ascending = False)[0:3] 
    
    # Convert them to frame
    points_place_year = points_place_year.to_frame()
    #Append the rest to this DF
    f1_df = f1_df.append(points_place_year)
    
f1_df = f1_df.rename(columns={"constructorId": "constructorid", "driverId": "driverid"})
f1_df = f1_df.reset_index()
f1_df 

,year,driverId,constructorId,forename,surname,nationality,points
0,1950,642,51,Nino,Farina,Italian,30.0
1,1950,786,51,Luigi,Fagioli,Italian,28.0
2,1950,579,51,Juan,Fangio,Argentine,27.0
3,1951,579,51,Juan,Fangio,Argentine,37.0
4,1951,647,6,Alberto,Ascari,Italian,28.0
...,...,...,...,...,...,...,...
199,2016,1,131,Lewis,Hamilton,British,380.0
200,2016,817,9,Daniel,Ricciardo,Australian,256.0
201,2017,1,131,Lewis,Hamilton,British,363.0
202,2017,20,6,Sebastian,Vettel,German,317.0


Import csv data

In [12]:
# Import data and convert it into DF
constr_1_csv = "./Resources/csv/constructor_results.csv"
constr_2_csv = "./Resources/csv/constructors.csv"
constr_3_csv = "./Resources/csv/races.csv"
constr_1_df = pd.read_csv(constr_1_csv)
constr_2_df = pd.read_csv(constr_2_csv)
constr_3_df = pd.read_csv(constr_3_csv)

# Merge the data in one
pre_contr_df = pd.merge(constr_1_df, constr_2_df, on="constructorId")
final_contr_df = pd.merge(pre_contr_df, constr_3_df, on="raceId")

# Filter only the needed columns
final_c1_df = final_contr_df[["constructorId","points","name_x","nationality","year"]]
final_c1_df = final_c1_df.rename(columns={"name_x": "constr_name"})

# Create an empty data frame
c1_df = pd.DataFrame()

# The goal of the project is to look at the top 3 best drivers for each year starting from 1950 to 2017
# We are only intersted to find the data since 1950 to 2017
for year in range(1950, 2018):
    c1_year = final_c1_df[final_c1_df["year"]==year]
    
    # Group by year, constructor ID,constr_name, nationality and points
    # take the sum of constructor points 
    # and then after sorting them, take the first 3 places
    pnts_place_year = c1_year.groupby(["year","constructorId","constr_name","nationality"]).sum()["points"]
    pnts_place_year  = pnts_place_year.sort_values(ascending = False)[0:3]
    
    # Convert them to frame
    pnts_place_year = pnts_place_year.to_frame()
    #Append the rest to this DF
    c1_df = c1_df.append(pnts_place_year)
    
c1_df = c1_df.reset_index()
c1_df = c1_df.rename(columns={"constructorId": "constructorid", "constr_name": "constrname"})
c1_df

,year,constructorid,constrname,nationality,points
0,1956,105,Maserati,Italian,28.0
1,1956,6,Ferrari,Italian,21.0
2,1956,128,Gordini,French,0.0
3,1958,6,Ferrari,Italian,61.0
4,1958,118,Vanwall,British,58.0
...,...,...,...,...,...
178,2016,9,Red Bull,Austrian,468.0
179,2016,6,Ferrari,Italian,398.0
180,2017,131,Mercedes,German,668.0
181,2017,6,Ferrari,Italian,522.0


In [13]:
# Connect to local data base
from sql_password import pgAdmin_pass
connect_str = f"postgres:{pgAdmin_pass}@localhost:5432/ETL" 
engine = create_engine(f"postgresql://{connect_str}")

In [14]:
# Check for tables
engine.table_names()

<ipython-input-14-73dc095f7130>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['c1_df', 'f1_df']

In [ ]:
# Use pandas to load sqlite converted DataFrame into Database
f1_df.to_sql(name="f1_df", con=engine, if_exists="append", index=False)

In [16]:
# Use pandas to load csv converted DataFrame into Database
c1_df.to_sql(name="c1_df", con=engine, if_exists="append", index=False)

In [17]:
# Confirm data has been added by querying f1_df table
pd.read_sql_query("select * from f1_df", con=engine)

,year,driverid,constructorid,forename,surname,nationality,points
0,1950,642,51,Nino,Farina,Italian,30.0
1,1950,786,51,Luigi,Fagioli,Italian,28.0
2,1950,579,51,Juan,Fangio,Argentine,27.0
3,1951,579,51,Juan,Fangio,Argentine,37.0
4,1951,647,6,Alberto,Ascari,Italian,28.0
...,...,...,...,...,...,...,...
199,2016,1,131,Lewis,Hamilton,British,380.0
200,2016,817,9,Daniel,Ricciardo,Australian,256.0
201,2017,1,131,Lewis,Hamilton,British,363.0
202,2017,20,6,Sebastian,Vettel,German,317.0


In [18]:
# Confirm data has been added by querying c1_df table
pd.read_sql_query("select * from c1_df", con=engine)

,year,constructorid,constrname,nationality,points
0,1956,105,Maserati,Italian,28.0
1,1956,6,Ferrari,Italian,21.0
2,1956,128,Gordini,French,0.0
3,1958,6,Ferrari,Italian,61.0
4,1958,118,Vanwall,British,58.0
...,...,...,...,...,...
178,2016,9,Red Bull,Austrian,468.0
179,2016,6,Ferrari,Italian,398.0
180,2017,131,Mercedes,German,668.0
181,2017,6,Ferrari,Italian,522.0
